In [ ]:
'''loading all the necessary libs'''
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
from time import sleep
import os




In [ ]:
"""Reading the files""" 
filepath = '../content/embarcacoes/'
df = pd.read_csv(filepath+'embacacoes.csv')  


In [ ]:
"""Select the boat trajectory and preparer the **M** matrix"""
N = 20 # selected trajectory
M = df.values # all the information matrix
pos = np.argwhere(M[:,12] == N) # vector of the position of points where the traject is the N
s = np.array([M[0:,7]]).T # speed vector

In [ ]:
"""Initiation the Gaussians to Find All the Probability Distributions"""
k = 2 #número de gaussianas
[N, Dim] = s.shape

aux = np.random.choice(N, N, replace=False)
w = s[aux[0:k],:]

for j in range(0,5):
    rot = np.array([])
    for n in range(0, N):
        rot = np.append(rot, np.argmin(sum(pow(np.ones((k,1))*s[n, :]-w, 2).T))  )
    for m in range(0, k):
        w[m,:] = np.mean(s[np.where(rot == m), :], axis=1)

del aux, j, n, m 

c = np.zeros((k,Dim,Dim))
for i in range(0, k):
    aux = s[np.where(rot == i)[0],:] - w[i, :]
    c[i,:,:] = np.dot(aux.T, aux)/N
del aux, i

## Clustering K-means Step

In [ ]:
#%matplotlib notebook
"""k-means clustering"""
#fig = plt.figure()
#ax = fig.add_subplot(111)
#plt.ion()
likehood = []
#fig.show()
#fig.canvas.draw()

for l in range(0,1):
    P = np.zeros((N,k))
    for j in range(0,N):
        for i in range(0,k):
            temp1 = 1/(pow(2*np.pi,Dim/2)*np.linalg.det(c[i,:, :]))
            temp2 = np.dot(s[j,:]-w[i,:], np.dot(np.linalg.inv(c[i,:,:]),(s[j,:]-w[i,:])))
            pi = 1/4*np.ones((k,1));
            P[j,i] = pi[i,0]*temp1*np.exp(-1/2*temp2)
    del temp1, temp2, i, j
    L = P
    den = np.sum(L, axis = 1)
    for j in range(0, k):
        for n in range(0, N):
            L[n,j] = P[n,j]/den[n]
    del n, j   
    Nk = np.sum(L, axis = 0)
    uknovo = np.zeros((w.shape))
    uknovo[0,:] = 1/Nk[0]*np.sum(L[:,0]*s.T,axis = 1)
    uknovo[1,:] = 1/Nk[1]*np.sum(L[:,1]*s.T,axis = 1)

    #ajustar covariância
    cknovo = np.zeros((k,Dim,Dim))
    for i in range(0, k):#len(u)):
        aux = s - uknovo[i, :]
        cknovo[i,:,:] = np.dot(L[:,i]*aux.T, aux)/Nk[i]

    del aux
    w = uknovo
    c = cknovo
    likehood = np.append(likehood,sum(np.log(np.sum(L, axis=1))))
    rot = np.zeros((N))
    for n in range(0, N):
        rot[n] = int(np.argmax(L[n,:])+1)

   
    #ax.clear()
    #ax.plot(s[np.where(rot == 1)[0], 0], '*b')
    #ax.plot(s[np.where(rot == 2)[0], 0], '*r')
       
    #fig.canvas.draw()


plt.plot(s[np.where(rot == 1)[0], 0],'*b', label="fish")
plt.plot(s[np.where(rot == 2)[0], 0],'*r', label="cruizing")
plt.legend()



In [ ]:
plt.plot(s[np.where(rot == 1)[0], 0],'*b', label="fish")
#plt.ylabel("Probability Distribution of Being Fishing")
plt.plot(s[np.where(rot == 2)[0], 0],'*r', label="cruizing")
#plt.ylabel("Probability Distribution of Being Cruzing")
plt.legend()


In [ ]:
pos_f = np.argwhere(rot == 1)
pos_c = np.argwhere(rot == 2)
Uc_t , Nc_t= np.unique(rot[pos_c[:-1,0]+1], return_counts = True)
Uf_t , Nf_t= np.unique(rot[pos_f[:-2,0]+1], return_counts = True)   
#%matplotlib widget
plt.plot(Uc_t, Nc_t/(np.sum(Nf_t)+np.sum(Nc_t)), '*r')
plt.plot(Uf_t, Nf_t/(np.sum(Nf_t)+np.sum(Nc_t)), '*b')
Nc_t/(+np.sum(Nc_t)),Nf_t/(np.sum(Nf_t))

a = np.array([Nf_t/(np.sum(Nf_t)),Nc_t/(+np.sum(Nc_t))])

In [ ]:
#%matplotlib widget 
"""probability distribuition just with k-means"""
t = np.linspace(0,21,211)
g1 = 1/(c[0]*np.sqrt(2*np.pi))*np.exp(-1/2*pow((t-w[0])/c[0], 2))
g2 = 1/(c[1]*np.sqrt(2*np.pi))*np.exp(-1/2*pow((t-w[1])/c[1], 2))
plt.plot(t,g1.T/np.sum(g1), 'r', label='fishing')
plt.plot(t,g2.T/np.sum(g2), 'b', label='cruizing')
plt.legend()

In [ ]:
b = np.array([g1[0,:].T/np.sum(g1), g2[0,:].T/np.sum(g2)])
b.shape
b = pd.DataFrame(b.T,columns=["cruzeiro","pesca" ])
b.to_csv (r'../content/distribution/b.csv', index = False, header=True, encoding='utf-8')
a = pd.DataFrame(a)
a.to_csv (r'../content/distribution/a.csv', index = False, header=True, encoding='utf-8')


## Markov Step

In [ ]:
filepath = '../content/distribution/'
a = pd.read_csv(filepath+'a.csv')#probabilidade de mudar de estado
b = pd.read_csv(filepath+'b.csv')#probabilidade dos eventos de velocidade
a = a.values
b = b.values
pi = np.array([0,1])#variavel da probabilidade 

In [ ]:
#%matplotlib notebook
#%load_ext autoreload
#%autoreload 2
from Markov import HMM
#importlib.reload(HMM)

fig = plt.figure()
ax1 = fig.add_subplot(211)
ax1.title.set_text('Proability Distribution')
ax2 = fig.add_subplot(212)
ax2.title.set_text('Error')
#plt.ion()
fig.show()
#fig.canvas.draw()

erro = np.array([]) 
for z in range(0,10):
    alpha, eta = HMM.forward(s, a, b, pi)
    beta = HMM.backward(s, b, a, eta)
    gamma = HMM.get_gamma(s, alpha, beta)    
    zeta = HMM.get_zeta(s, alpha, beta, eta, a, b)
    temp1 = np.sum(zeta, axis = 2)
    temp2 = np.sum(zeta, axis = 1)
    a_new = np.zeros((2,2))
    for i in range(0,2):
        for j in range(0,2):
            a_new[i, j] = temp1[i, j]/np.sum(temp1, axis = 1)[i]
    erro = np.append(erro, np.sum(np.abs(a-a_new)))
    a = np.copy(a_new)
    """find the probability distribution"""
    # mean of each distribution
    u1 = np.sum(gamma[0,:]*s[:,0])/np.sum(gamma[0,:])
    u2 = np.sum(gamma[1,:]*s[:,0])/np.sum(gamma[1,:])
    # covariance matrix of each distribution
    c1 = np.sum(gamma[0,:]*abs(s[:,0]-u1))/np.sum(gamma[0,:])
    c2 = np.sum(gamma[1,:]*abs(s[:,0]-u2))/np.sum(gamma[1,:]) 
    x = np.arange(0,21.1,0.1) #timeset
    
    g1 = 1/(c1*np.sqrt(2*np.pi))*np.exp(-1/2*pow((x-u1)/c1, 2))
    g2 = 1/(c2*np.sqrt(2*np.pi))*np.exp(-1/2*pow((x-u2)/c2, 2))
    
    b = np.array([g1/np.sum(g1),g2/np.sum(g2)]).T
    pi = gamma[:,0]  
   
    
  
    
    #ax1.clear()
   # ax2.clear()
  #  ax1.plot(x,b[:,0], 'r', label='cruizing')
 #   ax1.plot(x,b[:,1], 'b', label='fishing')
#    ax2.plot(erro)
#    fig.canvas.draw()
ax1.plot(x,b[:,0], 'r', label='cruizing')
ax1.plot(x,b[:,1], 'b', label='fishing')
ax2.plot(erro)
#   

In [ ]:
rot = np.zeros(len(gamma.T))
for i in range(0,len(gamma.T)):
    if(gamma[0,i] > gamma[1,i]) :
        rot[i] = 1;
    else: 
        rot[i] = 0
rot = np.array([rot])

In [ ]:
v = np.array([M[:,9], M[:,10], rot[0,:]])
pd.DataFrame(v[:,pos[:,0]])
pos = np.argwhere(M[:, -1] == 19)
v = v[:,pos[:,0]]

#%matplotlib notebook
plt.plot(v[0,np.where(v[2,:] == 1)].T, v[1,np.where(v[2,:] == 1)].T, '*b')
plt.plot(v[0,np.where(v[2,:] == 0)].T, v[1,np.where(v[2,:] == 0)].T, '*r')
plt.savefig("filepath.svg", format = 'svg', dpi=1000)


In [ ]:

#%matplotlib notebook
s = M[0:,7] #vetor de velocidade do trajeto N
s = np.array([s]).T
s = s[pos[:,0]]
plt.plot(s[np.where(v[2,:] == 1)], '*r')
plt.plot(s[np.where(v[2,:] == 0)], '*b')